In [ ]:
import pandas as pd
import json
from flask import Flask, render_template, jsonify
import plotly.express as px

app = Flask(__name__)

# Load CSV data
csv_file_path = "Cleaned_Final.csv"  # Make sure this path is correct
data = pd.read_csv(csv_file_path)

# Route to home page (Main Page)
@app.route('/')
def home():
    return render_template('main.html')

# # Route for Force-Directed Graph page
# @app.route('/fd.html')
# def force_directed_graph():
#     # Prepare the nodes and links based on your data structure
#     nodes = [{'id': str(country), 'group': 1} for country in data['country'].unique()]
    
#     # Create links (relationships) based on the data, ensuring no NaN values in 'new.sp'
#     links = [{'source': str(row['country']), 'target': str(row['g.whoregion']), 'value': row['new.sp']} 
#              for _, row in data.iterrows() if not pd.isna(row['new.sp'])]

#     # Return the nodes and links as JSON data
#     graph = {'nodes': nodes, 'links': links}
#     return jsonify(graph)

@app.route('/fd.html')
def force_directed_graph():
    # Render the HTML template for the force-directed graph
    return render_template('fd.html')

@app.route('/fd_data')
def force_directed_data():
    # Prepare the nodes (include both countries and regions)
    countries = data['country'].unique()
    regions = data['g.whoregion'].unique()
    
    # Ensure unique nodes list (countries + regions)
    nodes = [{'id': str(country), 'group': 1} for country in countries]
    nodes.extend([{'id': str(region), 'group': 2} for region in regions])
    
    # Create links (relationships) ensuring source and target exist in nodes
    links = []
    for _, row in data.iterrows():
        if pd.isna(row['new.sp']):
            continue

        # Check if source (country) and target (region) are in the nodes list
        source = str(row['country'])
        target = str(row['g.whoregion'])
        
        if source not in [node['id'] for node in nodes] or target not in [node['id'] for node in nodes]:
            continue  # Skip if either source or target is missing

        links.append({
            'source': source,
            'target': target,
            'value': row['new.sp']
        })

    # Return the nodes and links as JSON data
    return jsonify({'nodes': nodes, 'links': links})


# Other routes for additional visualizations
@app.route('/mc.html')
def map_chart():
    # Prepare the data for the map markers
    markers = []
    
    for _, row in data.iterrows():
        country_name = row['country']
        tot_newrel_value = row['tot.newrel']  # Using 'tot.newrel' for color coding
        
        # Skip rows where 'tot.newrel' is NaN
        if pd.isna(tot_newrel_value):
            continue
        
        markers.append({
            'country': country_name,
            'tot_newrel': tot_newrel_value
        })
    
    return render_template('mc.html', markers=markers)

# Function to categorize the color based on tot.newrel value
# def get_color(tot_newrel_value):
#     if tot_newrel_value < 100:
#         return 'green'  # Low cases
#     elif 100 <= tot_newrel_value < 500:
#         return 'orange'  # Medium cases
#     else:
#         return 'red'  # High cases

@app.route('/tv.html')
def timeline_visualization():
    # Render the HTML template for timeline visualization
    return render_template('tv.html')

@app.route('/tv_data')
def timeline_data():
    # Ensure the dataset includes the necessary 'g.whoregion' and 'year' columns, along with new.sp and tot.newrel
    required_columns = ['year', 'g.whoregion', 'new.sp', 'tot.newrel'] + list(data.select_dtypes(include=['number']).columns)
    filtered_data = data[required_columns].dropna(subset=['year', 'g.whoregion', 'new.sp', 'tot.newrel'])

    # Prepare the timeline data
    timeline_data = filtered_data.to_dict(orient='records')

    # Replace NaN values with None to ensure JSON validity
    timeline_data = [
        {key: (None if pd.isna(value) else value) for key, value in record.items()}
        for record in timeline_data
    ]

    # Get numeric columns for dropdown (including new.sp and tot.newrel)
    numeric_columns = ['new.sp', 'tot.newrel'] + data.select_dtypes(include=['number']).columns.tolist()

    return jsonify({'data': timeline_data, 'numeric_columns': numeric_columns})


# @app.route('/tv_data')
# def timeline_data():
#     # Ensure the dataset includes the necessary 'g.whoregion' and 'year' columns
#     required_columns = ['year', 'g.whoregion'] + list(data.select_dtypes(include=['number']).columns)
#     filtered_data = data[required_columns].dropna(subset=['year', 'g.whoregion'])

#     # Prepare the timeline data
#     timeline_data = filtered_data.to_dict(orient='records')

#     # Replace NaN values with None to ensure JSON validity
#     timeline_data = [
#         {key: (None if pd.isna(value) else value) for key, value in record.items()}
#         for record in timeline_data
#     ]

#     # Get numeric columns for dropdown
#     numeric_columns = data.select_dtypes(include=['number']).columns.tolist()

#     return jsonify({'data': timeline_data, 'numeric_columns': numeric_columns})

# @app.route('/tv.html')
# def timeline_visualization():
#     # Render the HTML template for timeline visualization
#     return render_template('tv.html')

# @app.route('/tv_data')
# def timeline_data():
#     # Prepare the timeline data from the dataset
#     timeline_data = data.to_dict(orient='records')

#     # Replace NaN values with None to ensure JSON validity
#     timeline_data = [
#         {key: (None if pd.isna(value) else value) for key, value in record.items()}
#         for record in timeline_data
#     ]

#     # Get numeric columns for dropdown
#     numeric_columns = data.select_dtypes(include=['number']).columns.tolist()

#     return jsonify({'data': timeline_data, 'numeric_columns': numeric_columns})


@app.route('/ht.html')
def hierarchical_tree_map():
    # Prepare data for the sunburst chart
    sunburst_data = []
    
    for _, row in data.iterrows():
        if not pd.isna(row['new.sp']):
            sunburst_data.append({
                'g.whoregion': row['g.whoregion'],
                'country': row['country'],
                'new.sp': row['new.sp'],
                'year': row['year']
            })
    
    # Return the sunburst data as JSON
    return render_template('ht.html', sunburst_data=sunburst_data)


# Route for the Sunburst Chart
# Sunburst Data Endpoint
@app.route('/sunburst_data')
def sunburst_data():
    # Create a hierarchy for the Sunburst chart from the dataset
    sunburst_data = {
        "name": "root",
        "children": []
    }

    # Group data by year, country, and region to create hierarchical levels
    for year in data['year'].unique():
        year_node = {"name": str(year), "children": []}
        
        # Filter data for the current year
        year_data = data[data['year'] == year]
        
        for region in year_data['g.whoregion'].unique():
            region_node = {"name": region, "children": []}
            
            # Filter data for the current region
            region_data = year_data[year_data['g.whoregion'] == region]
            
            for country in region_data['country'].unique():
                country_node = {
                    "name": country,
                    "value": region_data[region_data['country'] == country]['tot.newrel'].sum()
                }
                region_node['children'].append(country_node)
            
            year_node['children'].append(region_node)
        
        sunburst_data['children'].append(year_node)

    return jsonify(sunburst_data)

# Route for Sunburst Chart Page
@app.route('/sunburst.html')
def sunburst_chart():
    return render_template('sunburst.html')
# @app.route('/sunburst.html')
# def sunburst_data():
#     try:
#         # Load your CSV file
#         df = pd.read_csv('Cleaned_Final.csv')
        
#         # Generate a Sunburst Chart using Plotly
#         fig = px.sunburst(
#             df,
#             path=['g.whoregion', 'country', 'new.sp'],  # Update these based on your CSV file
#             color='new.sp',
#             title='Sunburst Chart for WHO Data'
#         )
        
#         # Convert the chart to HTML
#         chart_html = fig.to_html(full_html=False)
        
#         # Render the chart in the template
#         return render_template('sunburst.html', chart_html=chart_html)
    
#     except Exception as e:
#         return f"Error generating Sunburst Chart: {e}"
 
# Route for the dashboard
@app.route('/dashboard.html')
def dashboard():

    # Prepare data for the ht/sunburst chart
    sunburst_data = []
    
    for _, row in data.iterrows():
        if not pd.isna(row['new.sp']):
            sunburst_data.append({
                'g.whoregion': row['g.whoregion'],
                'country': row['country'],
                'new.sp': row['new.sp'],
                'year': row['year']
            })
    
    # Prepare the data for the map markers
    markers = []
    
    for _, row in data.iterrows():
        country_name = row['country']
        tot_newrel_value = row['tot.newrel']  # Using 'tot.newrel' for color coding
        
        # Skip rows where 'tot.newrel' is NaN
        if pd.isna(tot_newrel_value):
            continue
        
        markers.append({
            'country': country_name,
            'tot_newrel': tot_newrel_value
        })

    def get_color(tot_newrel_value):
        if tot_newrel_value < 100:
            return 'green'  # Low cases
        elif 100 <= tot_newrel_value < 500:
            return 'orange'  # Medium cases
        else:
            return 'red'  # High cases

    try:
        # Load your CSV file
        df = pd.read_csv('Cleaned_Final.csv')
        
        # Generate a Sunburst Chart using Plotly
        fig = px.sunburst(
            df,
            path=['g.whoregion', 'country', 'new.sp'],  # Update these based on your CSV file
            color='new.sp',
            title='Sunburst Chart for WHO Data'
        )
        
        # Convert the chart to HTML
        chart_html = fig.to_html(full_html=False)
        
    except Exception as e:
        return f"Error generating Sunburst Chart: {e}"
    
    # Return the sunburst data as JSON
    return render_template('dashboard.html', sunburst_data=sunburst_data, markers=markers, chart_html=chart_html)

if __name__ == '__main__':
    app.run(debug=True, port=3300, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:3300
Press CTRL+C to quit
